In [1]:
import findspark
findspark.init('/home/zeedlomo/spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('Linear_R').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
for item in data.head(2)[1]:
    print (item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

In [15]:
output = assembler.transform(data)

In [18]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [19]:
final_data = output.select('features', 'Yearly Amount Spent')

In [20]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
#tuple unpack to train-test split
train, test = final_data.randomSplit([0.7,0.3])

In [22]:
train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                343|
|   mean| 499.08880896991434|
| stddev|  81.06825312561065|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [23]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                157|
|   mean|  499.8060996981834|
| stddev|  75.59278340555787|
|    min|  314.4385182951061|
|    max|  765.5184619388373|
+-------+-------------------+



In [24]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [25]:
lr_model = lr.fit(train)

In [26]:
test_results = lr_model.evaluate(test)

In [28]:
#difference between the predicted value and the actual value from the test data
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.032867191419086|
|-12.901250185804827|
|  -8.43737918714487|
|  4.160876384225162|
| 1.9952338153809137|
|  2.326642435788983|
|  5.637986039732084|
|-0.7313224719717937|
|-7.0515373206989125|
|  3.196497108375354|
| -9.922886831606263|
|-11.686860932696277|
| -8.807182734170453|
|-17.553048857628767|
| 11.923897632163857|
|  4.280236364177995|
|0.35142472680962555|
| 18.457358311529333|
| 0.7966403456285889|
|  7.399626472483249|
+-------------------+
only showing top 20 rows



In [35]:
#Mean Absolute Error
test_results.meanAbsoluteError

7.6969664530360795

In [36]:
#Mean Squared Error
test_results.meanSquaredError

93.87166496282032

In [33]:
#Root Mean Squared Error (Most Popular - Shows how many units on average your model is off by)
test_results.rootMeanSquaredError

9.688739080129071

In [37]:
#R Squared Value (displays how much variance the model explains - the closer to 1 the better)
test_results.r2

0.9834671052397801

In [38]:
#Predicting on unlabeled data

unlabeled_data = test.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[31.0662181616375...|
|[31.1280900496166...|
|[31.3662121671876...|
|[31.4459724827577...|
|[31.5316044825729...|
|[31.6548096756927...|
|[31.6610498227460...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.8279790554652...|
|[31.8648325480987...|
|[31.8854062999117...|
|[31.9048571310136...|
|[31.9096268275227...|
|[31.9480174211613...|
|[32.0047530203648...|
|[32.0180740106320...|
|[32.0961089938451...|
|[32.1151190660142...|
+--------------------+
only showing top 20 rows



In [40]:
predictions = lr_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...| 451.7478750048108|
|[31.0662181616375...| 461.8345433934792|
|[31.1280900496166...| 565.6900659341995|
|[31.3662121671876...|426.42800617225976|
|[31.4459724827577...|482.88173111974766|
|[31.5316044825729...| 434.1889632935736|
|[31.6548096756927...|469.62543768781643|
|[31.6610498227460...|417.08967605187263|
|[31.7207699002873...| 545.8264707987219|
|[31.7366356860502...|493.73694914715657|
|[31.8279790554652...| 449.9256343785478|
|[31.8648325480987...|451.57814140950995|
|[31.8854062999117...|398.91045570664596|
|[31.9048571310136...| 491.5029062804449|
|[31.9096268275227...| 551.5221380410753|
|[31.9480174211613...| 457.6406405287198|
|[32.0047530203648...| 463.3945563938198|
|[32.0180740106320...|339.32575243378596|
|[32.0961089938451...| 374.6018150646146|
|[32.1151190660142...| 342.6585736913619|
+--------------------+------------